In [1]:
# Fine-tune GPT-3.5 Turbo on comments from posts on the subreddit r/AskReddit that are stored in the askreddit_comments.csv file.
# The model is fine-tuned using the Azure OpenAI API.
# The fine-tuned model should be able to genearate reddit comments that are similar to the comments in the askreddit_comments.csv file.
# The fine-tuned model uses posts that were generated by another fine-tuned model as the prompt.
# The generated posts that have been posted on the custom subreddit "r/DiplomskiRadFESB" are stored in the generated_posts.csv file, together with the post id.
# The generated posts are posted on the subreddit using the Reddit API.

from openai import AzureOpenAI
import os
from dotenv import load_dotenv

# Get the credentials from the environment
load_dotenv()

deployment_name = "gpt-35-turbo-0125-askreddit-comments"

# Initialize the Azure OpenAI client
client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-05-01-preview"  # This API version or later is required to access seed/events/checkpoint capabilities
)

# Upload the training dataset file to Azure OpenAI with the SDK
training_file_name = "askreddit_comments.jsonl"
validation_file_name = "askreddit_comments_validation.jsonl"

training_response = client.files.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
training_file_id = training_response.id

validation_response = client.files.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune"
)
validation_file_id = validation_response.id

print("Training file ID:", training_file_id)

Training file ID: file-9938ac9ae91240019ded301aff713aaf


In [2]:
# Start fine-tuning the model
response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model="gpt-35-turbo-0125"
)

# Motitor the fine-tuning job with job ID
job_id = response.id

print("Job ID:", job_id)
print("Job status:", response.status)
print(response.model_dump_json(indent=2))

Job ID: ftjob-211a39d689c94de38e030332d026c776
Job status: pending
{
  "id": "ftjob-211a39d689c94de38e030332d026c776",
  "created_at": 1724857213,
  "error": null,
  "fine_tuned_model": null,
  "finished_at": null,
  "hyperparameters": {
    "n_epochs": -1,
    "batch_size": -1,
    "learning_rate_multiplier": 1
  },
  "model": "gpt-35-turbo-0125",
  "object": "fine_tuning.job",
  "organization_id": null,
  "result_files": null,
  "seed": 1476439508,
  "status": "pending",
  "trained_tokens": null,
  "training_file": "file-9938ac9ae91240019ded301aff713aaf",
  "validation_file": "file-df9751847132499f9f89d779377ed8e8",
  "estimated_finish": null,
  "integrations": null
}


In [3]:
# Track training status

from IPython.display import clear_output
import time

start_time = time.time()

# Get the status of our fine-tuning job.
response = client.fine_tuning.jobs.retrieve(job_id)

status = response.status

# If the job isn't done yet, poll it every 10 seconds.
while status not in ["succeeded", "failed"]:
    time.sleep(10)

    response = client.fine_tuning.jobs.retrieve(job_id)
    print(response.model_dump_json(indent=2))
    print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
    status = response.status
    print(f'Status: {status}')
    clear_output(wait=True)

print(f'Fine-tuning job {job_id} finished with status: {status}')

# List all fine-tuning jobs for this resource.
print('Checking other fine-tune jobs for this resource.')
response = client.fine_tuning.jobs.list()
print(f'Found {len(response.data)} fine-tune jobs.')

Fine-tuning job ftjob-211a39d689c94de38e030332d026c776 finished with status: succeeded
Checking other fine-tune jobs for this resource.
Found 2 fine-tune jobs.


In [4]:
# Retrieve fine_tuned_model name

response = client.fine_tuning.jobs.retrieve(job_id)

print(response.model_dump_json(indent=2))
fine_tuned_model = response.fine_tuned_model

{
  "id": "ftjob-211a39d689c94de38e030332d026c776",
  "created_at": 1724857213,
  "error": null,
  "fine_tuned_model": "gpt-35-turbo-0125.ft-211a39d689c94de38e030332d026c776",
  "finished_at": 1724861872,
  "hyperparameters": {
    "n_epochs": 1,
    "batch_size": 9,
    "learning_rate_multiplier": 1
  },
  "model": "gpt-35-turbo-0125",
  "object": "fine_tuning.job",
  "organization_id": null,
  "result_files": [
    "file-4050c9d7d10d487282df95283ad1c649"
  ],
  "seed": 1476439508,
  "status": "succeeded",
  "trained_tokens": 1172033,
  "training_file": "file-9938ac9ae91240019ded301aff713aaf",
  "validation_file": "file-df9751847132499f9f89d779377ed8e8",
  "estimated_finish": null,
  "integrations": null
}


In [6]:
# Deploy the fine-tuned model

import json
import requests
import os

load_dotenv()

token = os.getenv("TEMP_AUTH_TOKEN")
subscription = os.getenv("AZURE_SUBSCRIPTION_ID")
resource_group = os.getenv("AZURE_RESOURCE_GROUP")
resource_name  = os.getenv("AZURE_OPENAI_RESOURCE_NAME")
model_deployment_name = deployment_name
fine_tuned_model = os.getenv("FINE_TUNED_MODEL_POST")

deploy_params = {'api-version': "2023-05-01"}
deploy_headers = {'Authorization': 'Bearer {}'.format(token), 'Content-Type': 'application/json'}

deploy_data = {
    "sku": {"name": "standard", "capacity": 1},
    "properties": {
        "model": {
            "format": "OpenAI",
            "name": fine_tuned_model,
            "version": "1"
        }
    }
}

deploy_data = json.dumps(deploy_data)   # Convert the data to a JSON string

request_url = f'https://management.azure.com/subscriptions/{subscription}/resourceGroups/{resource_group}/providers/Microsoft.CognitiveServices/accounts/{resource_name}/deployments/{model_deployment_name}'

print('Creating a new deployment...')

r = requests.put(request_url, params=deploy_params, headers=deploy_headers, data=deploy_data)

print(r)
print(r.reason)
print(r.json())

Creating a new deployment...
<Response [201]>
Created
{'id': '/subscriptions/868761f1-f48e-4aef-a9af-d397d405ff2d/resourceGroups/Diplomski_Rad/providers/Microsoft.CognitiveServices/accounts/DiplomskiRadJFP/deployments/gpt-35-turbo-0125-askreddit-comments', 'type': 'Microsoft.CognitiveServices/accounts/deployments', 'name': 'gpt-35-turbo-0125-askreddit-comments', 'sku': {'name': 'standard', 'capacity': 1}, 'properties': {'model': {'format': 'OpenAI', 'name': 'gpt-35-turbo-0125.ft-ba4c1bd78a6d4f39a9a0e6eac074982b', 'version': '1'}, 'versionUpgradeOption': 'NoAutoUpgrade', 'capabilities': {'maxContextToken': '16385', 'maxOutputToken': '4096', 'chatCompletion': 'true'}, 'provisioningState': 'Creating', 'rateLimits': [{'key': 'request', 'renewalPeriod': 10, 'count': 1}, {'key': 'token', 'renewalPeriod': 60, 'count': 1000}]}, 'systemData': {'createdBy': 'jprgin00@fesb.hr', 'createdByType': 'User', 'createdAt': '2024-08-28T18:48:18.9576979Z', 'lastModifiedBy': 'jprgin00@fesb.hr', 'lastModifie